In [2]:
# import packages
import pandas as pd
import numpy as np
# import the libraries
import datetime


import seaborn as sns # import seaborn for visualization of data
%matplotlib inline
import matplotlib.pyplot as plt#
import matplotlib as mpl
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
# Set figure size to (14,6)
plt.rcParams['figure.figsize'] = (14,6)

In [11]:
#import data csv

df = pd.read_csv('LTOTALNSA.csv', sep = ',',index_col='DATE',parse_dates = True)
df

,LTOTALNSA
DATE,
1976-01-01,864.600
1976-02-01,973.300
1976-03-01,1216.100
1976-04-01,1163.200
1976-05-01,1176.100
...,...
2021-11-01,1014.411
2021-12-01,1203.993
2022-01-01,989.560


In [12]:
# data exploration
df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 555 entries, 1976-01-01 to 2022-03-01
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   LTOTALNSA  555 non-null    float64
dtypes: float64(1)
memory usage: 8.7 KB


In [13]:
df.columns

Index(['LTOTALNSA'], dtype='object')

In [14]:
# Rename the column to make it more practical
df.rename(columns = {'LTOTALNSA' : 'weight'},inplace = True)

In [20]:
# See if data have missing values etc.
print(f'There are {df.isna().sum()[0]} missing values and {df.isnull().sum()[0]} zero values')

There are 0 missing values and 0 zero values


0